In [53]:
import pandas as pd 
import numpy as np 
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [54]:
data=pd.read_csv('data/tmdb_5000_credits.csv')
data_set=pd.read_csv('data/tmdb_5000_movies.csv')

In [55]:
data.columns = ['id','tittle','cast','crew']
data_set= data_set.merge(data,on='id')

In [56]:
vectorizer  = TfidfVectorizer(stop_words='english') # Comprobamos que el idioma con mayor porcentajes es en Inglés
data_set['overview'] = data_set['overview'].fillna('')
matrix = vectorizer.fit_transform(data_set['overview']) #Creamos la matriz
cosine_sim = linear_kernel(matrix, matrix)
indices = pd.Series(data_set.index, index=data_set['title']).drop_duplicates()
matrix.shape

(4803, 20978)

In [57]:
def get_recommendations(title, cosine_sim=cosine_sim):
    indice = indices[title] #Cogemos el índice que coincide con el título de la película.
    sim_scores = list(enumerate(cosine_sim[indice])) #Películas similares a la anterior.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #Ordenamos las películas en función de la puntuación.
    sim_scores = sim_scores[1:5] #Cogemos las 10 películas similares
    movie_indices = [i[0] for i in sim_scores] #Índices de las películas
    return df2['title'].iloc[movie_indices]

In [58]:
get_recommendations('Stolen')

1868    Cradle 2 the Grave
3905           Family Plot
3882                 Feast
2400            The Prince
Name: title, dtype: object

In [27]:
reader = Reader()
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [37]:
from surprise.model_selection import KFold
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x0000022D1DC20BC8>

In [42]:
svd = SVD()
model_selection.cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.89582796, 0.89724903, 0.88432386, 0.90425236, 0.89980291]),
 'test_mae': array([0.69138506, 0.69222878, 0.67848906, 0.69754209, 0.69216507]),
 'fit_time': (11.039252996444702,
  11.839588165283203,
  14.820144891738892,
  12.363412857055664,
  11.789721727371216),
 'test_time': (0.40192484855651855,
  0.248215913772583,
  0.5076420307159424,
  0.652778148651123,
  0.28409719467163086)}

In [43]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [45]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [47]:
svd.predict(1, 300, 3)

Prediction(uid=1, iid=300, r_ui=3, est=2.836783910340714, details={'was_impossible': False})